# Imports

In [1]:
import json
import gensim
from nltk.corpus import stopwords
import numpy as np
stop_words = set(stopwords.words('english'))
def clean_sentence(sentence):
    # Clean a sentence from unwanted characters
    sentence = sentence.replace('.','')
    sentence = sentence.replace(',','')
    sentence = sentence.replace('?','')
    sentence = sentence.replace('!','')
    sentence = sentence.replace(';','')
    sentence = sentence.replace(':','')
    sentence = sentence.replace('-','')
    return sentence

# Read data

In [2]:
with open('../../data/annotations/captions_train2014.json') as data_file:  
    captions = json.load(data_file)

# Bigrams

In [3]:
sentences = []

# Create list of sentences
for i in range(len(captions['annotations'])):
    sentence = captions['annotations'][i]['caption']
    sentence = clean_sentence(sentence)
    sentence = [word.lower() for word in sentence.split()]
    sentences.append(sentence)
    
# Train bigram
bigram_model = gensim.models.phrases.Phrases(sentences, min_count=4, threshold=200)

# word2vec

In [4]:
w2v_model = gensim.models.Word2Vec.load_word2vec_format('../../data/GoogleNews-vectors-negative300.bin.gz',binary=True)

# Create word2vec dictionary

In [8]:
word2vec = {}

for i in range(len(captions['annotations'])):
    if (i % 10000) == 0:
        print(i)
    # Find image filename
    image_id = captions['annotations'][i]['image_id']
    j, found = 0, 0
    while found == 0:
        if captions['images'][j]['id'] == image_id:
            file_name = captions['images'][j]['file_name']
            found = 1
        j += 1
    
    # Create bigrams from caption
    caption = clean_sentence(captions['annotations'][i]['caption'])
    caption = [word.lower() for word in caption.split()]
    caption = bigram_model[caption]
    
    # Remove unknown bigrams
    temp_bigram_sentence = []
    for j in range(len(caption)):
        if caption[j] not in w2v_model.vocab:
            temp_bigram_sentence += caption[j].split('_')
        else:
            temp_bigram_sentence.append(caption[j])
    caption = temp_bigram_sentence
    
    # Remove stop words in caption
    caption = [word for word in caption if word not in stop_words]
    
    # Create word2vec representation of caption
    w2v_vector = np.zeros(300)
    for word in caption:
        if word in w2v_model.vocab:
            w2v_vector += w2v_model[word]
    
    # Add w2v_vector to dictionary
    temp_list = word2vec.get(file_name,[])
    temp_list.append(w2v_vector)
    word2vec[file_name] = temp_list
        
    
    
    
    
    
        
    

0


/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
